In [55]:
mitos = ["Menggunakan kacamata atau lensa kontak dapat membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.", "Duduk terlalu dekat dengan layar komputer tidak merusak mata secara permanen.", "Menggunakan wortel secara berlebihan akan meningkatkan penglihatan dengan cepat.", "Bersepeda dapat menyebabkan kebutaan.", "Mata hanya perlu diperiksa jika ada masalah penglihatan.", "Menggunakan tetes mata secara berlebihan dapat menyebabkan ketergantungan.", "Membaca dalam cahaya redup tidak merusak mata secara permanen.", "Menggosok mata saat gatal dapat menyebabkan penyebaran infeksi.", "Istirahat sebentar setelah melihat layar tidak akan memperbaiki masalah penglihatan.", "Terlalu banyak membaca dalam cahaya rendah tidak akan merusak mata.", "Menyilangkan mata dapat menyebabkan mata juling.", "Menggunakan topeng tidur dapat menyebabkan kebutaan.", "Duduk terlalu dekat dengan lampu membaca dapat merusak penglihatan.", "Makanan pedas dapat merusak mata.",
"Menggunakan smartphone di gelap dapat merusak mata.", "Berenang setelah makan bisa menyebabkan rabun mata.", "Tidak perlu kacamata saat hujan karena air hujan membersihkan mata.", "Lensa kontak kolor dapat membuat mata lebih sensitif terhadap cahaya.", "Pilihan makanan yang salah dapat menyebabkan warna mata berubah.",
"Tidur dengan riasan mata dapat merusak bulu mata.", "Memakai kacamata minus bisa menyembuhkan mata minus.", "Melihat langsung matahari dapat meningkatkan penglihatan.",
"Memiringkan kepala saat membaca bisa merusak mata.", "Kaca mata hitam hanya diperlukan saat matahari terik.", "Tidak perlu melindungi mata di lingkungan berdebu atau berangin.", "Tidur cukup dapat menyembuhkan mata kering.", "Membaca di tempat gelap merusak penglihatan secara permanen.", "Kacamata membaca hanya diperlukan jika Anda sudah tua.", "Mengucek mata setelah berenang akan membuat penglihatan lebih baik.",
"Melihat TV dari dekat merusak mata.", "Air mata buatan dapat menyembuhkan semua masalah mata.", "Tidak memerlukan kacamata jika Anda hanya memiliki masalah penglihatan jauh.", "Melihat layar komputer sepanjang hari tidak akan merusak mata.", "Menggunakan kacamata minus bisa membuat mata semakin buruk.", "Berbicara saat angin berhembus dapat membuat mata merah.", "Lensa kontak tidak aman untuk digunakan saat berenang.", "Mata akan selalu memburuk seiring bertambahnya usia.", "Semua kacamata hitam memberikan perlindungan UV yang baik.", "Mata kering hanya terjadi pada orang tua.",
"Memakan ikan secara teratur bisa menghilangkan masalah mata.", "Minum air kelapa hijau dapat meningkatkan penglihatan.", "Bola mata berubah bentuk saat melihat benda dekat secara terus-menerus.", "Membaca dalam cahaya redup lebih baik daripada cahaya terang.",
"Melihat layar dengan sinar biru dapat merusak retina.", "Riasan mata yang tebal dapat melindungi mata dari sinar matahari.", "Kaca mata anti-silau akan merusak mata dalam jangka panjang.", "Merendam kantung teh di mata dapat menyembuhkan infeksi.", "Menggunakan kacamata minus jika mata terasa lelah dapat membantu.", "Menggunakan kacamata minus bisa menyebabkan mata lebih lelah.", "Menutup satu mata saat membaca dapat meningkatkan fokus.", "Tidak perlu kacamata matahari jika cuaca mendung.", "Makanan manis dapat menyebabkan astigmatisme.", "Tidur dengan lensa kontak dapat merusak kornea.", "Menggosok mata dengan daun mint dapat menghilangkan mata merah.", "Menggunakan kacamata pada anak-anak bisa membuat mata semakin buruk.", "Melihat TV dekat mata dapat memperbaiki penglihatan.", "Terlalu sering memakai kacamata minus bisa membuat mata malas.", "Membaca dengan cahaya lilin lebih baik untuk mata.", "Menggunakan lensa kontak saat tidur tidak masalah.", "Melihat sinar laser secara langsung tidak berbahaya bagi mata."]

fakta = ["Paparan sinar ultraviolet dari sinar matahari dapat menyebabkan kerusakan mata.", "Diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan.", "Perubahan penglihatan terjadi secara alami seiring bertambahnya usia.", "Merokok dapat meningkatkan risiko terjadinya katarak dan degenerasi makula.", "Konsumsi makanan bergizi, seperti makanan kaya vitamin A, C, E, dan mineral seperti seng, dapat membantu menjaga kesehatan mata.", "Istirahat reguler dan tetes mata bisa membantu mengurangi ketidaknyamanan mata kering.", "Penggunaan komputer dan perangkat elektronik lainnya dalam jangka waktu yang lama dapat menyebabkan Computer Vision Syndrome (CVS), menyebabkan gejala seperti mata kering, sakit kepala, dan ketegangan leher.", "Bentuk fisik mata setiap orang berbeda.", "Kacamata hitam yang baik melindungi mata dari sinar UV dan meminimalkan paparan sinar biru dari layar komputer atau perangkat elektronik lainnya.", "Deteksi dini adalah kunci untuk mencegah banyak masalah mata yang serius.", "Radang kelopak mata atau blepharitis adalah peradangan umum pada kelopak mata yang dapat menyebabkan mata merah, gatal, dan mengganggu penglihatan.", "Terlalu banyak paparan layar gadget dapat meningkatkan risiko mata kering.", "Glaukoma adalah kondisi di mana tekanan mata tinggi dapat merusak saraf optik dan menyebabkan kehilangan penglihatan.", "Usia di atas 40 meningkatkan risiko pengembangan katarak, yang mengaburkan lensa mata.", "Terlalu sering mengucek mata dapat memicu iritasi dan infeksi.",
"Kondisi mata seperti mata juling dapat diobati dengan terapi fisik atau operasi.", "Penggunaan lensa kontak yang tidak tepat atau kebersihan yang buruk dapat menyebabkan infeksi mata serius.", "Beberapa penyakit autoimun, seperti lupus dan rheumatoid arthritis, dapat mempengaruhi mata.", "Tekanan darah tinggi atau hipertensi dapat merusak pembuluh darah di mata.", "Konsumsi lemak sehat, seperti omega-3, dapat mendukung kesehatan mata.", "Lensa kontak tipe harian lebih higienis karena diganti setiap hari.", "Stres kronis dapat mempengaruhi kesehatan mata dan menyebabkan gejala mata kering.", "Alkohol dalam jumlah berlebihan dapat merusak saraf optik.", "Terlalu banyak mengonsumsi gula dapat meningkatkan risiko diabetes tipe 2, yang dapat merusak mata.",
"Beberapa obat, seperti antihistamin, dapat menyebabkan mata kering.", "Tekanan pada mata saat batuk atau bersin dapat memicu katarak pada orang yang rentan.", "Cedera mata dapat terjadi saat berolahraga atau kecelakaan, sehingga penting untuk menggunakan perlindungan mata yang tepat.", "Diabetes juga dapat menyebabkan pembuluh darah kecil di retina pecah, menyebabkan pendarahan dalam mata.", "Radang retina, atau uveitis, adalah kondisi yang dapat mengakibatkan mata merah, nyeri, dan penglihatan kabur.", "Lensa kontak tipe bulanan memerlukan perawatan dan pembersihan yang lebih intensif.", "Terlalu banyak konsumsi garam dapat meningkatkan risiko pembengkakan di sekitar mata.", "Terkena cahaya biru yang kuat dari layar komputer larut malam dapat mengganggu ritme sirkadian dan kualitas tidur.", "Mata kering dapat mempengaruhi kenyamanan selama penggunaan lensa kontak.", "Paparan sinar matahari yang berlebihan selama bertahun-tahun dapat menyebabkan pterygium atau ‘selaput mata’.", "Retina, bagian belakang mata, mengandung sel-sel fotoreseptor yang memungkinkan kita melihat cahaya dan warna.", "Terlalu banyak kafein dapat menyebabkan kelebihan cairan di mata, menyebabkan pembengkakan.", "Konsumsi vitamin C dan E dapat membantu mencegah degenerasi makula terkait usia.", "Pembengkakan kelopak mata (edema) dapat disebabkan oleh alergi atau masalah kesehatan lainnya.", "Beberapa kondisi seperti lupus dan rheumatoid arthritis dapat menyebabkan mata merah dan kering.", "Mata bayi baru lahir belum sepenuhnya berkembang dan perlu waktu untuk fokus dengan baik.", "Tidak semua masalah mata dapat dilihat dari luar; pemeriksaan mata profesional penting untuk deteksi dini.", "Mata manusia memiliki sekitar 107 juta sel batang yang membantu dalam penglihatan rendah cahaya.", "Pigmen pada iris mata mempengaruhi warna mata, dengan lebih banyak pigmen menghasilkan mata yang lebih gelap.", "Kondisi seperti hipotiroidisme dapat mempengaruhi kesehatan mata dan mengganggu produksi air mata.", "Pasokan darah yang buruk ke mata dapat menyebabkan masalah penglihatan dan kondisi serius seperti stroke mata.", "Pilihan kacamata atau lensa kontak yang tepat dapat membantu mengurangi mata lelah saat membaca atau bekerja di depan layar.", "Efek sinar biru pada retina dapat menyebabkan stres oksidatif yang merusak sel-sel mata.", "Kelopak mata memiliki kelenjar minyak yang dapat menyebabkan blepharitis jika terjadi infeksi atau penyumbatan.", "Kondisi jerawat rosasea juga dapat mempengaruhi mata, menyebabkan mata merah dan iritasi.", "Mata manusia dapat membedakan sekitar 10 juta warna berbeda.",
"Pembedahan katarak adalah prosedur yang umum untuk mengganti lensa mata yang kabur.",
"Terkena bahan kimia dapat menyebabkan luka kimia pada mata, yang dapat menyebabkan kerusakan permanen.", "Mata manusia dapat fokus pada objek yang berjarak mulai dari beberapa sentimeter hingga sejauh beberapa kilometer.", "Menerapkan kompres dingin dapat membantu mengurangi pembengkakan mata.", "Memijat area di sekitar mata dapat membantu meredakan ketegangan dan meningkatkan sirkulasi darah.", "Mata bayi cenderung berkedip lebih sedikit daripada orang dewasa.", "Pembuluh darah di bola mata membantu menyediakan nutrisi dan oksigen ke sel-sel mata.", "Radiasi sinar-X dapat merusak sel-sel di mata, itulah mengapa perlu menggunakan pelindung mata saat menjalani pemeriksaan sinar-X.", "Beberapa kondisi, seperti lupus, dapat memicu kerusakan saraf optik.", "Sel photoreceptor (sel batang dan kerucut) di retina mengubah cahaya menjadi sinyal saraf yang dikirimkan ke otak untuk pengolahan penglihatan."]


In [2]:
import gensim
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Conv1D, MaxPooling1D
from keras.models import load_model


In [4]:
# Definisi Function
def clean_and_lower(text):
    cleaned_text = ''.join(char.lower() for char in text if char.isalnum() or char.isspace())
    return cleaned_text

# Word2Vec Model
def word2vec_model(sentences, stopwords=None):
    words = [sentence.split() for sentence in sentences]
    model = gensim.models.Word2Vec(words, vector_size=100, window=5, min_count=1, workers=4)
    return model

# Convert sentences to average word vectors
def get_avg_word_vectors(sentences, model):
    avg_vectors = []
    for sentence in sentences:
        vectors = []
        for word in sentence.split():
            if word in model.wv:
                vectors.append(model.wv[word])
        if vectors:
            avg_vectors.append(np.mean(vectors, axis=0))
        else:
            avg_vectors.append(np.zeros(model.vector_size))
    return np.array(avg_vectors)

# Build and train a simple neural network
def build_model(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    # model.add(MaxPooling1D())
    # model.add(Conv1D(16, 16, 1, activation='relu'))
    # model.add(MaxPooling1D())
    # model.add(Conv1D(32, 32, 1, activation='relu'))
    # model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [58]:
# Kasih target setiap kalimat, 0 = mitos, 1 = fakta
labelledMitos = [(kalimat, 0) for kalimat in mitos]
labelledFakta = [(kalimat, 1) for kalimat in fakta]
print(labelledMitos[0])
print(labelledFakta[0])

('Menggunakan kacamata atau lensa kontak dapat membantu memperbaiki gangguan penglihatan dan mencegah masalah lebih lanjut.', 0)
('Paparan sinar ultraviolet dari sinar matahari dapat menyebabkan kerusakan mata.', 1)


In [59]:
# gabung fakta dan mitos, lalu shuffle
trainData = labelledMitos + labelledFakta
random.shuffle(trainData)
print(trainData[0])

('Tidur dengan lensa kontak dapat merusak kornea.', 0)


In [60]:
# preprocess, hilangkan yang bukan alpha numerik dan lower
cleanData = [(clean_and_lower(kalimat), label) for kalimat, label in trainData]
random.shuffle(cleanData)
print(cleanData[0])

('tidak perlu melindungi mata di lingkungan berdebu atau berangin', 0)


In [61]:
sentences = [data for data, label in cleanData]
labels = [label for data, label in cleanData]
print(len(sentences))
print(labels[0])

120
0


In [62]:
# Train & evaluate model
stopword = ["akan", "dapat", "dan", "dengan", "dari", "itu", "ini", "saya", "kamu", "dia", "mereka", "aku", "di", "ke"]
word2vec = word2vec_model(sentences, stopword) # ajarin model sentence word2vec
X_train_avg = get_avg_word_vectors(sentences, word2vec)
model = build_model(input_dim=X_train_avg.shape[1])
model.fit(X_train_avg, np.array(labels), batch_size=5, epochs=20, verbose=1)

Epoch 1/20
24/24 [==============================] - 1s 2ms/step - loss: 0.6943 - accuracy: 0.5000
Epoch 2/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 3/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 4/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5000
Epoch 5/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5083
Epoch 6/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6897 - accuracy: 0.5083
Epoch 7/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6883 - accuracy: 0.5000
Epoch 8/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6848 - accuracy: 0.5167
Epoch 9/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6796 - accuracy: 0.6083
Epoch 10/20
24/24 [==============================] - 0s 2ms/step - loss: 0.6748 - accuracy: 0.5750
Epoch 11/20
24/24 [

In [63]:
# Save model
model.save("w2v.h5")
word2vec.save("w2v.bin")

In [10]:
# Coba hasil
loaded_model = load_model("w2v.h5")
w2vmodel = gensim.models.Word2Vec.load("w2v.bin")

# Now you can use the loaded_model for predictions
new_sentences = ['Melihat layar komputer dengan dekat dapat tidak merusak mata secara permanen', 
                 'Diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan',
                 'dfaskfjasdjfasmdcasonsn kdcsanco dndsnfoinasdnf ndjnacznjdnsia ewjnwfndds icdo']
clean_new_sentences = [clean_and_lower(kalimat) for kalimat in new_sentences]
new_vectors = get_avg_word_vectors(clean_new_sentences, w2vmodel)
predictions = loaded_model.predict(new_vectors)

print("prediksi", predictions)
print("=====================================================")

for sentence, prediction in zip(clean_new_sentences, predictions):
    print(sentence, prediction)
    # print(f"Sentence: {sentence} - Sentiment: {sentiment}")

1/1 [==============================] - 0s 110ms/step
prediksi [[0.02100241]
 [0.8110154 ]
 [0.562873  ]]
melihat layar komputer dengan dekat dapat tidak merusak mata secara permanen [0.02100241]
diabetes dapat menyebabkan retinopati diabetik yang dapat mengganggu penglihatan [0.8110154]
dfaskfjasdjfasmdcasonsn kdcsanco dndsnfoinasdnf ndjnacznjdnsia ewjnwfndds icdo [0.562873]
